In [1]:
import numpy as np
import pandas as pd
import sqlalchemy as sa
import pickle, json, requests, base64


## Build a scikit-learn model

In [2]:
from sklearn import datasets

iris = datasets.load_iris()
X = iris.data  
Y = iris.target
# print(iris.DESCR)

In [7]:
from sklearn.svm import OneClassSVM

random_state = np.random.RandomState(0)
clf = OneClassSVM(random_state=random_state)

clf.fit(X, Y)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma='auto', kernel='rbf',
      max_iter=-1, nu=0.5,
      random_state=<mtrand.RandomState object at 0x7f4cc7649678>,
      shrinking=True, tol=0.001, verbose=False)

In [8]:
#clf.__dict__

## Generate SQL Code from the Model

In [11]:

def test_ws_sql_gen(pickle_data):
    WS_URL="http://192.168.88.88:1888/model" # "https://sklearn2sql.herokuapp.com/"
    b64_data = base64.b64encode(pickle_data)
    data={"Name":"model1", "PickleData":b64_data , "SQLDialect":"postgresql"}
    r = requests.post(WS_URL, json=data)
    content = r.json()
    # print(content)
    lSQL = content["model"]["SQLGenrationResult"][0]["SQL"]
    return lSQL;


In [12]:
pickle_data = pickle.dumps(clf)
lSQL = test_ws_sql_gen(pickle_data)
print(lSQL[0:4000])

WITH "SV_data" AS 
(SELECT "Values".sv_idx AS sv_idx, "Values".dual_coeff AS dual_coeff, "Values".sv_0 AS sv_0, "Values".sv_1 AS sv_1, "Values".sv_2 AS sv_2, "Values".sv_3 AS sv_3 
FROM (SELECT 0 AS sv_idx, 1.0 AS dual_coeff, 4.9 AS sv_0, 3.0 AS sv_1, 1.4 AS sv_2, 0.2 AS sv_3 UNION ALL SELECT 1 AS sv_idx, 1.0 AS dual_coeff, 4.7 AS sv_0, 3.2 AS sv_1, 1.3 AS sv_2, 0.2 AS sv_3 UNION ALL SELECT 2 AS sv_idx, 1.0 AS dual_coeff, 4.6 AS sv_0, 3.1 AS sv_1, 1.5 AS sv_2, 0.2 AS sv_3 UNION ALL SELECT 3 AS sv_idx, 1.0 AS dual_coeff, 5.4 AS sv_0, 3.9 AS sv_1, 1.7 AS sv_2, 0.4 AS sv_3 UNION ALL SELECT 4 AS sv_idx, 1.0 AS dual_coeff, 4.6 AS sv_0, 3.4 AS sv_1, 1.4 AS sv_2, 0.3 AS sv_3 UNION ALL SELECT 5 AS sv_idx, 1.0 AS dual_coeff, 4.4 AS sv_0, 2.9 AS sv_1, 1.4 AS sv_2, 0.2 AS sv_3 UNION ALL SELECT 6 AS sv_idx, 0.38507940877 AS dual_coeff, 4.9 AS sv_0, 3.1 AS sv_1, 1.5 AS sv_2, 0.1 AS sv_3 UNION ALL SELECT 7 AS sv_idx, 1.0 AS dual_coeff, 5.4 AS sv_0, 3.7 AS sv_1, 1.5 AS sv_2, 0.2 AS sv_3 UNION ALL SEL

## Execute the SQL Code

In [13]:
# save the dataset in a database table

#engine = sa.create_engine('sqlite://' , echo=False)
engine = sa.create_engine("postgresql://db:db@localhost/db?port=5432", echo=False)
conn = engine.connect()

lTable = pd.DataFrame(X);
lTable.columns = ['Feature_0', 'Feature_1', 'Feature_2', 'Feature_3']
lTable['KEY'] = range(lTable.shape[0])
lTable.to_sql("INPUT_DATA" , conn,   if_exists='replace', index=False)


In [14]:
sql_output = pd.read_sql(lSQL , conn);
sql_output = sql_output.sort_values(by='KEY').reset_index(drop=True)

In [15]:
sql_output.sample(12, random_state=1960)

,KEY,AnomalyScore,OutlierIndicator
114,114,-1.734906,-1
74,74,0.482320,1
9,9,0.000087,1
88,88,0.406374,1
25,25,0.211161,1
5,5,-1.338805,-1
48,48,-0.376297,-1
117,117,-10.866101,-1
83,83,1.774387,1
105,105,-7.460889,-1


In [17]:
sql_output.OutlierIndicator.value_counts()

 1    75
-1    75
Name: OutlierIndicator, dtype: int64

## Scikit-learn Prediction

In [18]:
skl_outputs = pd.DataFrame()
skl_output_key = pd.DataFrame(list(range(X.shape[0])), columns=['KEY']);
skl_output_anomaly_score = pd.DataFrame(clf.decision_function(X), columns=['AnomalyScore']);
skl_output_is_outlier = pd.DataFrame(clf.predict(X), columns=['OutlierIndicator'])
skl_output = pd.concat([skl_output_key, skl_output_anomaly_score, skl_output_is_outlier] , axis=1)
skl_output.sample(12, random_state=1960)


,KEY,AnomalyScore,OutlierIndicator
114,114,-1.734906,-1.0
74,74,0.482320,1.0
9,9,0.000087,1.0
88,88,0.406374,1.0
25,25,0.211161,1.0
5,5,-1.338805,-1.0
48,48,-0.376297,-1.0
117,117,-10.866101,-1.0
83,83,1.774387,1.0
105,105,-7.460889,-1.0


## Comparing the SQL and Scikit-learn Predictions

In [19]:
sql_skl_join = skl_output.join(sql_output , how='left', on='KEY', lsuffix='_skl', rsuffix='_sql')

In [20]:
sql_skl_join.sample(12, random_state=1960)

,KEY_skl,AnomalyScore_skl,OutlierIndicator_skl,KEY_sql,AnomalyScore_sql,OutlierIndicator_sql
114,114,-1.734906,-1.0,114,-1.734906,-1
74,74,0.482320,1.0,74,0.482320,1
9,9,0.000087,1.0,9,0.000087,1
88,88,0.406374,1.0,88,0.406374,1
25,25,0.211161,1.0,25,0.211161,1
5,5,-1.338805,-1.0,5,-1.338805,-1
48,48,-0.376297,-1.0,48,-0.376297,-1
117,117,-10.866101,-1.0,117,-10.866101,-1
83,83,1.774387,1.0,83,1.774387,1
105,105,-7.460889,-1.0,105,-7.460889,-1


In [22]:
condition = (sql_skl_join.OutlierIndicator_sql != sql_skl_join.OutlierIndicator_skl)
sql_skl_join[condition]


,KEY_skl,AnomalyScore_skl,OutlierIndicator_skl,KEY_sql,AnomalyScore_sql,OutlierIndicator_sql
